# Alice, Bob & Eve: Digital signatures using quantum-safe ML-DSA

The algorithms mentioned in previous parts (RSA, ECDSA) are not quantum-safe. Once Eva will have quantum computer available, she can forge the signatures. Luckily, we already have two post-quantum digital signature algorithms available. Here we'll take a look at the ML-DSA (Module‑Lattice‑based Digital Signature Algorithm), formerly known as CRYSTALS-Dilithium.

This algorithm is available natively in .NET 10. For older versions you can use the `Microsoft.Bcl.Cryptography` package. We will be actually doing this, because current version of .NET Interactive is using .NET 9.

## Hack: Installing the NuGet package 

In regular .NET 10 applications you do not need to install this NuGet package, as everything is part of the runtime itself. However, at time of writing, .NET Interactive (which is used in notebooks) uses version 9.0 and we have to import the package manually:

In [2]:
// Install the Microsoft.Bcl.Cryptography package 
#r "nuget: Microsoft.Bcl.Cryptography"

// Import namespace
using System.Security.Cryptography;

// Show what runtime we're actually on
Console.WriteLine($"Runtime version:  {System.Runtime.InteropServices.RuntimeInformation.FrameworkDescription}");

// Check whether ML-DSA is supported on this platform
Console.WriteLine($"ML-DSA supported: {MLDsa.IsSupported}");

Installed Packages Microsoft.Bcl.Cryptography, 10.0.1

Runtime version:  .NET 9.0.11
ML-DSA supported: True


## Alice

As usual, Alice will generate her key pair. Notice that the keys are significantly longer not only than ECDSA, but also than RSA keys. But this is completely different scheme, so it's quite hard to compare them.

The API is kind of similar to the one exposed by `AsymmetricAlgorithm` base class, but the post-quantum algorithm classes are not derived from this class, because the philosophy and usage is different enough.

The ML-DSA is available in three variants: ML-DSA-44, ML-DSA-65 and ML-DSA-87. The numbers are arbitrary; they are sequential numbers of original CRYSTALS-Dilithium parameter sets and do not directly relate to any specific parameters, such as key sizes.

Higher number is "better" in terms of security (and "worse" in terms of performance and data size). The following table shows the three variants and their parameters (security roughly equivalent to AES with given key lenghs and size of various aspects):

| Variant | AES Equivalent Length | Public Key Length (bytes) | Public Key Length (bits) | Private Key Length (bytes) | Private Key Length (bits) | Signature Length (bytes) | Signature Length (bits) |
|--------|---------------:|--------------------:|-------------------:|---------------------:|--------------------:|-------------------:|------------------:|
| MLDSA44 | 128 | 1312 | 10496 | 2528 | 20224 | 2420 | 19360 |
| MLDSA65 | 192 | 1952 | 15616 | 4000 | 32000 | 3293 | 26344 |
| MLDSA87 | 256 | 2592 | 20736 | 4864 | 38912 | 4595 | 36760 |

For this demo we will be using the simplest variant, ML-DSA-44:

In [3]:
using System.Security.Cryptography;

var mldsa = MLDsa.GenerateKey(MLDsaAlgorithm.MLDsa44);
var privateKeyBytes = mldsa.ExportMLDsaPrivateKey();
var publicKeyBytes = mldsa.ExportMLDsaPublicKey();


// Display the keys
Console.WriteLine($"Alice's Private Key ({privateKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(privateKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Private Key (20480 bits):
IoGy0mGViLLGWbgU2Jnn6vRozFV1t7txf1ni0/GiTWKXcW0it8hZwo8h08K4O2g49sVgdklb36zN
qa4WAJiKHWp3q5g6pgLw+V/EfuhaWcxGhtYggVPtGjfqRfXyc3ZnjLJB8XqbWCFrbmlDXvt08gR6
MaDs/l7YuVRPqzCLShMLmQQMuCiaOAkSsQ0Ms5BLiBEaoAzcki1RxkVUBiXQGADUyDABOUqDhgzZ
mA0aR2LAyHHLmIQTORKJRkmkCCnaBmwkp0FhAgXKlEBcQEAQlUECwpCKMHECRZIAEYqJpnBMlkCk
lmQLSQ5IQEYTE2yRFA4DhyliyIlaJI7MxCwUJyrEoGEEA0AIl0AbOSAEwxAch1EZiYQhGUkBF5GK
IozCCJFJCGFCFIkRBwbDMgKQxHFStmHQpAALIG0EBIUKNErCuGDcIEjApEgAiAGTuGHAgEXLxJHb
MIILRABZQAwZhYyhBhDIyEyagGUDIkEKGSrEqGgSAgVbhGQDIgYKMxJYhhFYMgGgJgBapCXCoGEa
IgHcmIGimHBBxoHTAHHhGGFAAmgcuJERBIWBIoQAtCERM5AkQyHbxmESJEBKIiJhpAFcJCEUto1k
xgQjpUAUKI6RqCAaB2DSNnJQsg0JADAMQDEKGWnYqAHDSHCTMEzcmEQAsgAUIGoTtYkERI5JQAAK
MGrRkkiDJmiMmAnBqCRblAEYJ0HMKAwiwUwRiZFcwhEZR3AMgCkRQSXMJgIRoQEatgSQFk0RsEwb
OZAMQoEYAGhYSJLMOHECAWbhImDKmCzJNClJko1IqERREIVTOIlhGAJQRnKgpA0KKEHLNJEjEk0B
RAHRSFGimEEgqITiCCQTSYmYGAkaEpCJRoaDQojZiEUhkwDitJHiQHAJRmxgRDAZJwEJiCmZgGBk
yInQQIbBAmSRgnFbsoxgyAxgEA0IEE4aRQigFGyiBm

In [4]:
Console.WriteLine($"Alice's Public Key ({publicKeyBytes.Length * 8} bits):");
Console.WriteLine(Convert.ToBase64String(publicKeyBytes, Base64FormattingOptions.InsertLineBreaks));

Alice's Public Key (10496 bits):
IoGy0mGViLLGWbgU2Jnn6vRozFV1t7txf1ni0/GiTWKe4ibNXLr3y0F6EYhvVfVtzAV/alpi0RVg
2RTn2gKQXNI7YZBWIT6degHYBRFEW/GMKkYUWXhNDGcuJNdUA3iwvKjWprOGIJJCHjafcSgL6wyL
xOKy2HMQmzowgAnaYk1JNDWaiEJ8QLG4+Lv5l6WCsNYy8uSNAn1/GxHuJCmxuzgmnJdaN7rzabPu
ZXJjvPC/kB3PaEijI8TuRnCg64j/utB5r+YsT27JoiN8Y1gc9e844bGVp+eLA8He5UWHDwFJvH5u
boO0cfHHn6fkTYjFTD+RP8tMqQVaN+QDqcB+2GP5Rxo+cNyLfZT3F38+BB6XKpMAXX+m4o51Rs/T
a7nxrdwHTqwmDV0AvobDH1GqpLfCk0mD3eOFY2REbOT55SljXVQ5GuoxF52ejOae77Ve/+GNHcN9
Wg5iNyRH1NdCgP2OM7mCTbUZ6LAPr/tK1UVkXPwRqTTPzfGkSVU99BUfRE+/bSCKP5E0gg3J+Umh
EBj4kEML014SXD+xsRofNZyAKq04oDH7GU6g0fVFMhUEnMyIJXVgboOmm3djk7WEAZ6C8andwXNt
dUO87RyUWJWDvFAAtHYt10m8Vgnz6tIjpexF+qL9NtyUFkLET/DTwPM3q53e48U4mFmjoSYEKp4/
olkvLzUSkNCMlXXuHRneVm3gyhxoEtFQrOeYBzSgIaxvEIjJJFrhoDiy0Oh+Zg4vCry5pSoOBXKw
ntVz2Auj0e6wFIc95D9bHI0o647gE3kQlidTR+3YO67hFrxlV2ZC/YLaTMC0NT7WCmu+PJpUKaLQ
vKlKLl5bdPV5UJIR0u4PbRKlMR6o5lBfWp1hper+9zxTGA1CKCdp9jmblNllupamC+H8chJHovX9
Tg4G1r0eCMAY8p7Nw2BejvT32nlgF2Rus4DaYiCkxaY

Alice will then generate signature of her message. Notice that the signature itself is also larger than DSA or RSA:

In [5]:
var messageText = "Bob, I love you!";
var messageBytes = System.Text.Encoding.UTF8.GetBytes(messageText);
Console.WriteLine($"Original Message: {messageText}");
Console.WriteLine();

// Sign the message
var signatureBytes = mldsa.SignData(messageBytes);
var signatureText = Convert.ToBase64String(signatureBytes, Base64FormattingOptions.InsertLineBreaks);
Console.WriteLine($"Signature:");
Console.WriteLine(signatureText);

Original Message: Bob, I love you!

Signature:
1kxXD7bvx+28kmVD5OizdyMwqEAUA5kdR4+FsQ10Jd5E1Vk29O9qvBF6H2u035ykqfej0v3xTiPs
E1Q8VW0djymYIE9ytfYBsebckrBK8MawD0DEGxEY8L5NGPAvpJRv3cNL1CJGkqdcljELgWX6Eku+
3QIib1GLkIvpeslBxxyRdBe1Kv1eR/SpjLxwlRYfhooZaoZImSg+q+Syk7FMNmJL6oVAoWVGPjjB
HoxofUfaMYVwRF0jEsWlpp05QcW9fxmcSCfhQ2UVpn/x7JfqHh8CDDLr3lMkj1BVzlGRuCRoGhs8
6XPz3+p7H3/zjQ3nAagOHA7MkKgrF0SZ/2R3rwKosKKbYlVj28clY7lC38bE6DtpPT4gKyV8/BNB
3R/3RC65KWjhS9LGLT0yIqMwj+UnajvzoEQfixrHDw0MxB1M8p3rElZCd0UdcMn00uM1chFPIn/0
05UjA+M+jeU09XNCFY8w0VktZIHdfcsPs/a6Y2PEaqcpbXN3uZ8whWSzHlYi1z1DuXnbiLZKbOnc
sz7amuPLBskDi//Dr8X8kPfol38AvENrUoq9AmpRfw3tM2REEO6/iT/QRZUx9Arubc/C+BQ0zTJc
3yx3gHWS6i2eYUv358UqoRJUDegEYYI8sSrQyowfWO3lAZKgnaU4ZmS6V4nlCNAClGC3+RkssCTk
Lo4ji5PMX8uqMYWdJU2tFgM8sGvQyOz1fpeUvLXP7PLPPedfMvi3dRsmj8wLns6U0q/6EQnQxinE
pzVA+W2eCxtlOxg2ORgWiY5EEQcPLSRIcUGwoHnAPq26dw5Qgx15vwpHZUefTwumFmX8D2XOTHu8
yabDDUwRsgMemQ6d+Td4E/wed4kM5HTMUYxvAEJKWqPIDtkqDtI7TgAwBQn1a5H49rXgeWY9izNg
Go7a4l//lm87KxIZjlOAXqeoC4fjE

## Bob

Bob will verify the signature in a fashion similar to RSA or DSA:

In [6]:
// Function to verify signature
bool VerifySignature(byte[] publicKey, string message, byte[] signature) {
    // Create MLDsa instance for verification using the public key
    using var mldsaVerify = MLDsa.ImportMLDsaPublicKey(MLDsaAlgorithm.MLDsa44, publicKey);

    // Convert message to bytes
    var messageBytesVerify = System.Text.Encoding.UTF8.GetBytes(message);

    // Verify the signature
    return mldsaVerify.VerifyData(messageBytesVerify, signature);
}

Console.WriteLine($"Is 'Bob, I love you!' valid? {VerifySignature(publicKeyBytes, "Bob, I love you!", signatureBytes)}");
Console.WriteLine($"Is 'Bob, I hate you!' valid? {VerifySignature(publicKeyBytes, "Bob, I hate you!", signatureBytes)}");

Is 'Bob, I love you!' valid? True
Is 'Bob, I hate you!' valid? False
